# *Скрейпване на PCBuild*

### *Нужни библиотеки*

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

### *Построяване на линка*

In [2]:
base_url = 'https://pcbuild.bg'
category_url = f'/c-pc-komponenti/c-video-karti/page='

### *Вземане на броя на страниците, които ще скрейпваме*

In [3]:
headers = {'User-Agent' : "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"}
response = requests.get(base_url + category_url, headers = headers)
soup = BeautifulSoup(response.text, 'lxml')
ul_list = soup.find('ul', class_ = 'paginator list-unstyled')
last_li = ul_list.find_all('li')[-1]
a_tag = last_li.find('a')
title = a_tag.get('title')
last_page = title.strip('Страница ')
print(last_page)

10


### *Скрейпваме линковете на продуктите от всички страници*

In [4]:
product_links = []
page_list = list(range(1, pd.to_numeric(last_page)+1))

for page in page_list:
    target_url = f'{base_url}{category_url}{page}'
    response = requests.get(target_url, headers = headers)
    soup = BeautifulSoup(response.text, 'lxml')

    products_div = soup.find('div', class_ = 'products-list')
    
    for article in products_div.find_all("article", class_="product-item"):
        a_tag = article.find("a", class_="img-container")
        if a_tag and a_tag.has_attr("href"):
            product_links.append(base_url + a_tag["href"])

print(f"Намерени {len(product_links)} броя линкове.")

Намерени 227 броя линкове.


### *Обхождане на всеки намерен линк и събиране на информация от него*

In [5]:
product_data = []
missing_skus = []

for url in product_links:
    product_resp = requests.get(url, headers = headers)
    product_soup = BeautifulSoup(product_resp.text, "lxml")

    td = product_soup.find('td', string='Продуктов код')
    if td:
        code_td = td.find_next_sibling('td')
        if code_td:
            product_code = code_td.get_text(strip=True)
        else:
            missing_skus.append(url)
            continue
    else:
        missing_skus.append(url)
        continue

    div_price = product_soup.find('div', class_='price')
    price = div_price.contents[0].strip()
    div_cents = div_price.find('span', class_ = 'cents')
    cents = div_cents.contents[0].strip()

    full_price = float(f"{price}.{cents}")

    product_data.append({
        'SKU' : product_code,
        'Price_pcbuild' : full_price
    })


print(f"{len(missing_skus)} SKU-та не са описани на страниците на продуктите!")

55 SKU-та липсват от страниците на продуктите!


### *Създаване на DataFrame от данните*

In [6]:
df = pd.DataFrame(product_data)
df.head()

,SKU,Price_pcbuild
0,1A1-G00425900H,1689.00
1,NE7507T019T2-GB2031U,1756.00
2,NE75060S19P1-GB2062D,675.00
3,NE75060S19P1-GB2063D,678.72
4,NE75060T19P1-GB2063S,678.72


### *Записване на данните в .csv формат*

In [7]:
df.to_csv('pcbuild_gpu_sku_prices.csv', index = False)